
# Images → Foam half-life (baseline)
Quick baseline that pairs the converted foam stacks with tabular features to predict the foam half-life (as defined in the converter: first time the height or BC hits half the max).
- Loads NSID `.h5` files from `../data/foam`
- Builds a compact feature table (bubble metrics + image summary stats)
- Trains a scikit-learn random forest regressor


In [ ]:

from pathlib import Path

DATA_DIR = Path("../data/foam").resolve()
h5_files = sorted(DATA_DIR.glob("*.h5"))
if not h5_files:
    raise FileNotFoundError("No .h5 files in ../data/foam. Run scripts/convert_foam_to_h5.py first.")

# Pick one dataset to start; switch index if desired
H5_PATH = h5_files[0]
print("Using", H5_PATH)


In [ ]:

import numpy as np
import pandas as pd
from PIL import Image
from SciFiReaders import NSIDReader
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Load NSID content
reader = NSIDReader(H5_PATH)
datasets = reader.read()
im_ds = datasets["Channel_000"]
feat_ds = datasets["Channel_001"]

feature_names = [
    f.decode() if isinstance(f, (bytes, bytearray)) else f
    for f in feat_ds.metadata.get("feature_names", [])
]
X = pd.DataFrame(feat_ds.compute(), columns=feature_names)

images = im_ds.compute().astype(np.float32) / 255.0
# Lightweight image summary stats
flat = images.reshape(images.shape[0], -1)
X["intensity_mean"] = flat.mean(axis=1)
X["intensity_std"] = flat.std(axis=1)

label_val = float(im_ds.original_metadata.get("half_life_s", np.nan))
y = np.full(len(X), label_val, dtype=np.float32)

print(f"Frames: {len(X)}, Pixel size (mm): {im_ds.original_metadata.get('pixel_size_mm')}")
print("Half-life target (every frame uses the dataset value):", label_val)


In [ ]:

# Train/test split + model
train_cols = [c for c in X.columns if c != "half_life_s"]
X_train, X_test, y_train, y_test = train_test_split(
    X[train_cols], y, test_size=0.2, random_state=42
)

model = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
        ("rf", RandomForestRegressor(n_estimators=150, random_state=42)),
    ]
)
model.fit(X_train, y_train)
preds = model.predict(X_test)
mae = mean_absolute_error(y_test, preds)
print(f"MAE (s): {mae:.3f}")
print("Dataset-wide predicted half-life (mean over frames):", model.predict(X[train_cols]).mean())


In [ ]:

import matplotlib.pyplot as plt

rf = model.named_steps["rf"]
importances = rf.feature_importances_
order = np.argsort(importances)[::-1][:10]

plt.barh(range(len(order)), importances[order][::-1])
plt.yticks(range(len(order)), [train_cols[i] for i in order][::-1])
plt.xlabel("Feature importance")
plt.title("Top predictors of half-life")
plt.show()
